<a href="https://colab.research.google.com/github/SatellogicTeam/GOESDetection/blob/main/GOESDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalación de librerías (AWS S3 File System y NetCDF4)

In [9]:
%%capture
!pip install s3fs
!pip install netCDF4

### Importación de librerías.

In [10]:
import s3fs
import numpy as np
import os
from netCDF4 import Dataset
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from os import listdir
from os.path import isfile, join
from itertools import chain 

!mkdir archivos
%cd /content/archivos

mkdir: cannot create directory ‘archivos’: File exists
[Errno 2] No such file or directory: '/content/archivos'
/content/fotos


### Obtener las coordenadas en base a la proyección y "Mask", que contiene información de incendios

In [11]:
  def lat_lon_reproj(dataset):

    # designate dataset
    g16nc = dataset

    # GOES-R projection info and retrieving relevant constants
    proj_info = g16nc.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis

    # grid info
    lat_rad_1d = g16nc.variables['x'][:]
    lon_rad_1d = g16nc.variables['y'][:]

    # create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

    # lat/lon calc routine from satellite radian angle vectors

    lambda_0 = (lon_origin*np.pi)/180.0

    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)

    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

    # latitude and longitude projection for plotting data on traditional lat/lon maps
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    mask = g16nc.variables['Mask'][:]
    return lon,lat,mask

### Creación de diccionario filtrando valores no importantes

In [12]:
%%time

def crear_dict(lat, lon,mask):
  incendios={}
  for i in chain(range(10,16), range(30, 36)):
    incendios[str(i)] =[]
    for pos,lista in enumerate(mask):
      for m in np.where(mask[pos] == i):
        if len(m)>0:
          incendios[str(i)].append((lat[pos][m[0]],lon[pos][m[0]]))
  return incendios

CPU times: user 62 µs, sys: 0 ns, total: 62 µs
Wall time: 66.3 µs


### Descargar archivos del Bucket S3 del satélite GOES-16 y GOES-17

In [13]:
fs = s3fs.S3FileSystem(anon=True)

path_archivos_aws=["s3://noaa-goes16/ABI-L2-FDCF/2021/068/16/OR_ABI-L2-FDCF-M6_G16_s20210681650193_e20210681659501_c20210681700023.nc","s3://noaa-goes17/ABI-L2-FDCF/2021/068/19/OR_ABI-L2-FDCF-M6_G17_s20210681910321_e20210681919387_c20210681919484.nc"]

for i in path_archivos_aws:
  nombre_archivo= i[10:16] + "_" + i[-17:]
  fs.get(i,nombre_archivo)



### Crear lista con nombres de los archivos descargados

In [14]:
onlyfiles = [f for f in listdir() if isfile(join(f))]

### Crea una lista de diccionarios de incendios

In [15]:
incendios=[]
for i in onlyfiles:
  dataset=Dataset(i)
  lon, lat,mask = lat_lon_reproj(dataset)
  incendios.append(crear_dict(lat,lon,mask))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in sqrt


### Asignar colores y mostrar los incendios en el mapa

In [22]:
map = folium.Map([0,0], zoom_start=5, tiles='cartodbpositron')
color_dict = {
    "10":'red',"30":'red',
    "11":'lightred',"31":'lightred',
    "12":'lightblue',"32":'lightblue',
    "13":'darkred',"33":'darkred',
    "14":'pink',"34":'pink',
    "15":"orange","35":'orange'
}

# 10 : Processed fire pixel
# 11 : Saturated fire pixel
# 12 : Cloud contaminated fire pixel
# 13 : High probability fire pixel
# 14 : Medium probability fire pixel
# 15 : Low probability fire pixel
# 30 : Temporally Filtered Processed fire pixel
# 31 : Temporally Filtered Saturated fire pixel
# 32 : Temporally Filtered Cloud contaminated fire pixel
# 33 : Temporally Filtered High probability fire pixel
# 34 : Temporally Filtered Medium probability fire pixel
# 35 : Temporally Filtered Low probability fire pixel 

for incendio in incendios:
  for key in incendio.keys():
    for coordenadas in incendio[key]:

      lat, lon = coordenadas
      Circle(location=[lat,lon], radius=500, color=color_dict[key]).add_to(map)
 
map

Página de referencia de valores de "Mask" en página 43: https://www.star.nesdis.noaa.gov/goesr/documents/ATBDs/Baseline/ATBD_GOES-R_FIRE_v2.6_Oct2013.pdf